In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import matplotlib.pyplot as plt

# ML
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)

# Deep Learning (BiLSTM)
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# SHAP
import shap

# Notebook settings
pd.set_option("display.max_columns", None)

INJURY_DATA_PATH = "injury_data.csv"
NFL_DATA_PATH = "nfl_injuries_2013_2023.csv"

TARGET_COL_TABULAR = "injury"  # Cross-sport target
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15


In [2]:
def nfl_week_to_date(season, week):
    """
    Approximates NFL Week 1 as September 1st of the season year.
    """
    base = datetime(season, 9, 1)
    return base + timedelta(days=(week - 1) * 7)

def load_sportA(path):
    df = pd.read_csv(path)

    df = df.rename(columns={
        "Player_Age": "age",
        "Player_Weight": "weight",
        "Player_Height": "height",
        "Previous_Injuries": "prev_injuries",
        "Training_Intensity": "workload",
        "Recovery_Time": "recovery_time",
        "Likelihood_of_Injury": "injury"
    })

    df["sport"] = "synthetic"

    # Pseudo-date
    base_date = datetime(2010, 1, 1)
    df["date"] = [base_date + timedelta(days=i) for i in range(len(df))]

    df["player_id"] = ["SYN_" + str(i) for i in range(len(df))]

    keep_A = [
        "player_id", "date", "sport", "injury", "workload",
        "age", "weight", "height", "prev_injuries", "recovery_time"
    ]
    
    return df[keep_A]


def load_sportB(path):
    df = pd.read_csv(path)

    df["player_id"] = df["gsis_id"]
    df["date"] = df.apply(
        lambda r: nfl_week_to_date(r["season"], r["week"]), axis=1
    )

    df["injury"] = df["report_status"].isin(["Out", "Doubtful"]).astype(int)

    practice_map = {
        "Full Participation in Practice": 1.0,
        "Limited Participation in Practice": 0.5,
        "Did Not Participate In Practice": 0.0
    }
    df["workload"] = df["practice_status"].map(practice_map)

    df["sport"] = "NFL"

    keep_B = [
        "player_id", "date", "sport", "injury", "workload",
        "team", "position", "season", "week",
        "report_status", "practice_status"
    ]

    return df[keep_B].sort_values(["player_id", "date"])

In [3]:
def load_all():
    dfA = load_sportA("D://0 GIM Studies//MLBA//project//injury_data.csv") # Modify according to path 
    dfB = load_sportB("D://0 GIM Studies//MLBA//project//nfl_injuries_2013_2023.csv") # Modify according to path 
    return pd.concat([dfA, dfB], ignore_index=True)

df = load_all()


In [4]:
def compute_days_since_last_injury(df):
    df = df.sort_values(["player_id", "date"])
    df["days_since_last_injury"] = np.nan

    for pid, sub in df.groupby("player_id"):
        last_date = None
        for idx, row in sub.iterrows():
            df.loc[idx, "days_since_last_injury"] = np.nan if last_date is None else (row["date"] - last_date).days
            if row["injury"] == 1:
                last_date = row["date"]

    return df

def compute_rolling_features(df):
    df = df.sort_values(["player_id", "date"])

    df["rolling_injuries_4w"] = (
        df.groupby("player_id")["injury"]
        .rolling(window=5, min_periods=1).sum()
        .reset_index(level=0, drop=True)
    )

    df["rolling_workload_4w"] = (
        df.groupby("player_id")["workload"]
        .rolling(window=5, min_periods=1).mean()
        .reset_index(level=0, drop=True)
    )

    return df

def compute_injury_history(df):
    df = df.sort_values(["player_id", "date"])
    df["injuries_last_90d"] = np.nan

    for pid, sub in df.groupby("player_id"):
        idx = sub.index
        for i in range(len(sub)):
            current_date = sub.iloc[i]["date"]
            past = sub[(sub["date"] >= current_date - timedelta(days=90)) &
                       (sub["date"] < current_date)]
            df.loc[idx[i], "injuries_last_90d"] = past["injury"].sum()

    return df

def compute_future_injury(df):
    df = df.sort_values(["player_id", "date"])
    df["future_injury_next_week"] = 0

    for pid, sub in df.groupby("player_id"):
        idx = sub.index
        for i in range(len(sub)):
            current = sub.iloc[i]["date"]
            next_week = current + timedelta(days=7)
            future = sub[(sub["date"] > current) & (sub["date"] <= next_week)]["injury"].any()
            df.loc[idx[i], "future_injury_next_week"] = int(future)

    return df

In [5]:
def engineer_all(df):
    df = compute_days_since_last_injury(df)
    df = compute_injury_history(df)
    df = compute_rolling_features(df)
    df = compute_future_injury(df)
    return df

df_eng = engineer_all(df)
df_eng.head()

,player_id,date,sport,injury,workload,age,weight,height,prev_injuries,recovery_time,team,position,season,week,report_status,practice_status,days_since_last_injury,injuries_last_90d,rolling_injuries_4w,rolling_workload_4w,future_injury_next_week
1000,00-0000585,2013-09-01,NFL,1,0.0,NaN,NaN,NaN,NaN,NaN,DEN,CB,2013.0,1.0,Out,Did Not Participate In Practice,NaN,0.0,1.0,0.000000,1
1001,00-0000585,2013-09-08,NFL,1,0.0,NaN,NaN,NaN,NaN,NaN,DEN,CB,2013.0,2.0,Out,Did Not Participate In Practice,7.0,1.0,2.0,0.000000,0
1002,00-0000585,2013-09-15,NFL,0,0.5,NaN,NaN,NaN,NaN,NaN,DEN,CB,2013.0,3.0,Questionable,Limited Participation in Practice,7.0,2.0,2.0,0.166667,0
1003,00-0000585,2013-09-22,NFL,0,0.5,NaN,NaN,NaN,NaN,NaN,DEN,CB,2013.0,4.0,Questionable,Limited Participation in Practice,14.0,2.0,2.0,0.250000,0
1004,00-0000585,2013-09-29,NFL,0,0.5,NaN,NaN,NaN,NaN,NaN,DEN,CB,2013.0,5.0,Questionable,Limited Participation in Practice,21.0,2.0,2.0,0.300000,0


In [6]:
def time_split(df, target):
    df = df[~df[target].isna()]
    df = df.sort_values("date")

    n = len(df)
    t1 = int(n * TRAIN_RATIO)
    t2 = int(n * (TRAIN_RATIO + VAL_RATIO))

    return df.iloc[:t1], df.iloc[t1:t2], df.iloc[t2:]

df_train, df_val, df_test = time_split(df_eng, TARGET_COL_TABULAR)
(len(df_train), len(df_val), len(df_test))

(41757, 8948, 8948)

In [7]:
def clean_dataframe(df):
    # Strip whitespace from all object columns
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.strip()

    # Replace bad tokens
    BAD = set(["nan", "NaN", "NAN", "None", "NONE", "NULL", "", " ", "\r", "\n", "\r\n", "\r\n    "])
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].apply(lambda x: "Unknown" if x in BAD else x)

    return df

# Apply to all splits after feature engineering
df_train = clean_dataframe(df_train.copy())
df_val   = clean_dataframe(df_val.copy())
df_test  = clean_dataframe(df_test.copy())


In [9]:
def prepare_tabular(df_train, df_val, df_test, target):
    drop_cols = ["player_id", "date"]

    numeric = [c for c in df_train.columns if df_train[c].dtype != 'object' and c not in drop_cols + [target]]
    categorical = [c for c in df_train.columns if c not in numeric + drop_cols + [target]]

    pre = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric),
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical)
        ]
    )

    X_train = pre.fit_transform(df_train[numeric + categorical])
    X_val   = pre.transform(df_val[numeric + categorical])
    X_test  = pre.transform(df_test[numeric + categorical])

    y_train = df_train[target].values
    y_val   = df_val[target].values
    y_test  = df_test[target].values

    feature_names = numeric + list(pre.named_transformers_["cat"].get_feature_names_out(categorical))

    return X_train, y_train, X_val, y_val, X_test, y_test, pre, feature_names

In [10]:
def train_models(X_train, y_train):
    models = {
        "LogReg": LogisticRegression(max_iter=1000, class_weight="balanced"),
        "RandomForest": RandomForestClassifier(
            n_estimators=300, class_weight="balanced", random_state=42),
        "XGBoost": XGBClassifier(
            n_estimators=400, learning_rate=0.05, max_depth=5,
            subsample=0.8, colsample_bytree=0.8, eval_metric="logloss")
    }
    for m in models.values():
        m.fit(X_train, y_train)
    return models

In [ ]:
def evaluate(model, X, y):
    y_pred = model.predict(X)
    try:
        y_proba = model.predict_proba(X)[:,1]
    except:
        y_proba = y_pred.astype(float)

    return {
        "Accuracy": accuracy_score(y, y_pred),
        "Precision": precision_score(y, y_pred, zero_division=0),
        "Recall": recall_score(y, y_pred, zero_division=0),
        "F1": f1_score(y, y_pred, zero_division=0),
        "ROC-AUC": roc_auc_score(y, y_proba) if len(set(y)) > 1 else np.nan
    }, confusion_matrix(y, y_pred)

results_tab = {}
cms_tab = {}

for name, model in models_tab.items():
    metrics, cm = evaluate(model, X_test, y_test)
    results_tab[name] = metrics
    cms_tab[name] = cm
    print("\n======", name, "======")
    print(metrics)
    print(cm)


In [ ]:
X_sample = X_test[:500].toarray() if hasattr(X_test, "toarray") else X_test[:500]

for name in ["RandomForest", "XGBoost"]:
    model = models_tab[name]
    explainer = shap.TreeExplainer(model)
    shap_vals = explainer.shap_values(X_sample)
    if isinstance(shap_vals, list):
        shap_vals = shap_vals[1]

    # Global
    imp = np.abs(shap_vals).mean(axis=0)
    df_imp = pd.DataFrame({"feature": feature_names, "importance": imp})
    df_imp.sort_values("importance", ascending=False).head(20)
    display(df_imp.head(20))

    # Local example
    shap.initjs()
    shap.force_plot(explainer.expected_value, shap_vals[0], feature_names)

In [ ]:
def build_nfl_sequences(df, seq_len=8):
    nfl = df[df["sport"] == "NFL"].copy()
    nfl = nfl.sort_values(["player_id", "date"])

    feature_cols = [c for c in nfl.columns if c not in ["player_id", "date", "future_injury_next_week"]]

    # One-hot categorical
    cat = [c for c in feature_cols if nfl[c].dtype == 'object']
    num = [c for c in feature_cols if c not in cat]

    nfl = pd.get_dummies(nfl, columns=cat)
    feature_cols = [c for c in nfl.columns if c not in ["player_id", "date", "future_injury_next_week"]]

    X_seq = []
    y_seq = []

    for pid, sub in nfl.groupby("player_id"):
        sub = sub.sort_values("date")
        Xmat = sub[feature_cols].values
        y = sub["future_injury_next_week"].values

        if len(sub) < seq_len:
            continue

        for i in range(len(sub) - seq_len + 1):
            X_seq.append(Xmat[i:i+seq_len])
            y_seq.append(y[i+seq_len-1])

    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq = build_nfl_sequences(df_eng, seq_len=8)
len(X_seq)

In [ ]:
def train_bilstm(X, y):
    if len(X) == 0:
        print("Not enough NFL sequence data.")
        return None, None

    n = len(X)
    train_end = int(n * 0.7)
    val_end = int(n * 0.85)

    X_train, X_val, X_test = X[:train_end], X[train_end:val_end], X[val_end:]
    y_train, y_val, y_test = y[:train_end], y[train_end:val_end], y[val_end:]

    model = models.Sequential([
        layers.Input(shape=(X.shape[1], X.shape[2])),
        layers.Bidirectional(layers.LSTM(64)),
        layers.Dropout(0.3),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    es = callbacks.EarlyStopping(patience=5, restore_best_weights=True)

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=64, verbose=0, callbacks=[es])

    y_proba = model.predict(X_test).ravel()
    y_pred = (y_proba >= 0.5).astype(int)

    metrics = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, zero_division=0),
        "Recall": recall_score(y_test, y_pred, zero_division=0),
        "F1": f1_score(y_test, y_pred, zero_division=0),
        "ROC-AUC": roc_auc_score(y_test, y_proba) if len(set(y_test)) > 1 else np.nan
    }

    return metrics, model


bilstm_metrics, bilstm_model = train_bilstm(X_seq, y_seq)
bilstm_metrics


In [ ]:
print("\n===== TABULAR MODEL PERFORMANCE =====")
for m, res in results_tab.items():
    print("\n", m)
    print(res)

print("\n===== BILSTM (NFL ONLY) =====")
print(bilstm_metrics)

print("\nPipeline complete.")